In [2]:
# Find a route from the first node to the second node in a directed graph.

def find_route(node1, node2):
  found_path = None
  queue = Queue()
  node = node1
  node.shortest_path = [node]
  all_visited_nodes = [node]
  while node:
    for adjacent in node.adjacency_list:
      if not adjacent.shortest_path:
        adjacent.shortest_path = node.shortest_path + [adjacent]
        if adjacent == node2:
          found_path = node.shortest_path + [adjacent]
          break
        queue.add(adjacent)
        all_visited_nodes.append(adjacent)
    node = queue.remove()
  for visited in all_visited_nodes:
    visited.shortest_path = None
  return found_path
  
class Node():
  def __init__(self, data, adjacency_list=None):
    self.data = data
    self.adjacency_list = adjacency_list or []
    self.shortest_path = None
  
  def add_edge_to(self, node):
    

In [ ]:
import json
import pymongo
from flask import Flask, request, jsonify
# from flask_cors import CORS

from collections import Counter
import math

app = Flask(__name__)
CORS(app)

#reading configs
with open(r'/home/shubham/JavaScript/side-project-shubham0831/config.json') as json_file:
    config = json.load(json_file)

username = config['username']
password = config['password']
database = config['database']

#connecting to db
uri = "mongodb+srv://" +username+":"+password+"@"+database+".tjeuf.mongodb.net/RecipeSearch?retryWrites=true&w=majority"
client = pymongo.MongoClient(uri)
db = client.Recipes
recipe_collection = db.Recipe

@app.route('/',methods = ['POST', 'GET'])
def login():
    if (request.method == 'POST'):
        # print("got post request")
        # print(request.json)
        body = request.json

        #doing list(set(list())) to remove duplicates
        avail_ingredients = set(body['ingredients'])
        dbResults = list(recipe_collection.find({ "ingredients": {"$in": list(avail_ingredients) } }
                         , {"name" : 1, "_id" : 0, "ingredients" : 1, "ingredient_quantity" : 1, "instructions" : 1, "nutrition_values" : 1}))
        
        response = {}
        response['total_match'] = []
        response['potential_match'] = []
        for recipes in dbResults:
            required_ingredients = set(recipes['ingredients'])
            if (avail_ingredients == required_ingredients):
                response['total_match'].append(recipes)
                continue
            difference = abs(len(required_ingredients), len(avail_ingredients))

            if (difference <= 2):
                response['potential_match'].append(recipes)
        
        
        #if we have too many potential matched, we just send the first 5
        if (len (response['potential_match']) > 5):
            response['potential_match'] = response['potential_match'][:5]
        
        
        return "<h1> POST </h1>"
    else :
        return "<h1> GET </h1>"

def getRecipes (ingredient_list : list):
    print("in all recipes")
    print(f"ingredient list is {ingredient_list}")
    cursor = recipe_collection.find({ "ingredients": {"$in": ingredient_list } }
                         , {"name" : 1, "_id" : 0, "ingredients" : 1, "ingredient_quantity" : 1, "instructions" : 1, "nutrition_values" : 1})
    all_recipes = list(cursor)

    #perfect match is the list of recipes for which the user has all the available ingredients
    #probably match is the list of recipes for which the user has at-least 70-80% of the required ingredients
    perfect_match, probable_match = splitRecipes (ingredient_list, all_recipes)
    # print(all_recipes[0])
    return perfect_match, probable_match


#https://stackoverflow.com/questions/14720324/compute-the-similarity-between-two-lists
def splitRecipes (source_list : list, all_recipes : list):
    perfect_match = []
    probable_match = []
    for recipe in all_recipes:
        #doing list(set(list())) to remove duplicates
        recipe_ingredient_list = set(recipe['ingredients'])
        so
        

    return perfect_match, probable_match

# https://stackoverflow.com/questions/14720324/compute-the-similarity-between-two-lists
def cosine_similarity (c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)


a = Counter([ "sugars", "oil", "egg substitute", "orange juice", "leavening agents", "wheat flour" ])

b = Counter([ "sugars", "oil", "egg substitute", "orange juice", "leavening agents", "wheat flour" ])

print(cosine_similarity(a,b))